# Creating blood vessels network in 3D via a 3D matrix vector representation.
The matrix is stored as a sparse matrix via a hashmap (dict) to save memory.
This matrix represent vectors. Multiple nonzero close vectors pointing to the same direction represent a flow like a vessel.

In [ ]:
from functools import reduce

import numpy as np
from sparray import sparray
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

In [ ]:
def extend_vessel(m, x: int, y: int, z: int, distance: int, submatrix_shape: tuple):
    """Extend a vessel in the matrix m at the point (x, y, z)

    The point (x, y, z) must be a nonzero point in the matrix m.
    It will edit the next nonzero point in the direction of the vector + the direction of the sum of all the neighbors vectors in a submatrix of shape=(X, X, X) and the euclidean distance between each of them.
    This function will edit the matrix m in place to extend the vessel.
    """
    # Get the vector at the point (x, y, z)
    vector = m[x, y, z]
    # Get the submatrix of shape (X, X, X) centered at the point (x, y, z)
    submatrix = m.virtual_projection()[x - submatrix_shape[0] // 2: x + submatrix_shape[0] // 2,
                y - submatrix_shape[1] // 2: y + submatrix_shape[1] // 2,
                z - submatrix_shape[2] // 2: z + submatrix_shape[2] // 2]
    # submatrix is an array of (X, 3) where the second dimension is the vector
    # Get the sum of all the vectors in the submatrix using fold
    sum_vector = reduce(lambda v1, v2: (v1[0] + v2[0], v1[1] + v2[1], v1[2] + v2[2]), submatrix.get_data())

    # Get the next point in the direction of the vector + the direction of the sum of all the neighbors vectors
    next_point = (vector[0] + sum_vector[0], vector[1] + sum_vector[1], vector[2] + sum_vector[2])

    # Sum of the relative vector between x, y, z and each point in the submatrix
    # PS: This part is the most time-consuming, It's why, maybe an r-tree could be used to speed up the process
    rotation = [0, 0, 0]
    for i in range(x - submatrix_shape[0] // 2, x + submatrix_shape[0] // 2):
        for j in range(y - submatrix_shape[1] // 2, y + submatrix_shape[1] // 2):
            for k in range(z - submatrix_shape[2] // 2, z + submatrix_shape[2] // 2):
                if m[i, j, k] != (0, 0, 0):
                    rotation[0] += x - i
                    rotation[1] += y - j
                    rotation[2] += z - k
    norm_rotation = np.linalg.norm(rotation)
    norm_sum_vector = np.linalg.norm(sum_vector)
    # Normalize the rotation vector
    rotation = (rotation[0] / (norm_rotation + 1) * norm_sum_vector,
                rotation[1] / (norm_rotation + 1) * norm_sum_vector,
                rotation[2] / (norm_rotation + 1) * norm_sum_vector)

    next_point = (next_point[0] + rotation[0],
                  next_point[1] + rotation[1],
                  next_point[2] + rotation[2])

    # Normalize the next point with the distance (sphere)
    norm = np.linalg.norm(next_point)
    next_point = (next_point[0] / norm * distance, next_point[1] / norm * distance, next_point[2] / norm * distance)

    # Add a bit of randomness to the next
    next_point = (next_point[0] + np.random.uniform(-2, 2), next_point[1] + np.random.uniform(-2, 2),
                  next_point[2] + np.random.uniform(-2, 2))

    # # Add a little curve to point to the center of the matrix
    # next_point = (min(0.5, next_point[0] + (5000 - x) / 100),
    #               min(0.5, next_point[1] + (5000 - y) / 100),
    #               min(0.5, next_point[2] + (5000 - z) / 100))

    # discretize the next point
    next_point = (int(next_point[0]), int(next_point[1]), int(next_point[2]))

    coord_next_point = (x + next_point[0], y + next_point[1], z + next_point[2])

    # set the point vector value
    m[coord_next_point] \
        = (next_point[0] / distance, next_point[1] / distance, next_point[2] / distance)

    # Return the next point
    return coord_next_point

Plot the parse matrix

In [ ]:
def plot_matrix(m, distance: int):
    items = m.get_items()  # list of tuples (coords, vector)
    plt.close('all')
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    for item in items:
        x, y, z = item[0]  # coords
        u, v, w = item[1]  # vector value
        # also add colors depending on the vector
        ax.quiver(x, y, z, u, v, w, length=distance, normalize=True,
                  color=((distance + u) / 10, (distance + v) / 10, (distance + w) / 10))
    plt.show()

In [ ]:
def creat_vessel(m, x: int, y: int, z: int, distance: int, submatrix_shape: tuple | None, length: int):
    """Create a vessel in the matrix m at the beginning point (x, y, z)

    The point (x, y, z) must be a nonzero point in the matrix m.
    If the submatrix_shape is None, the submatrix shape will be computed automatically (log(length) * 4 + 1),
    more the length is big, more the submatrix shape will be big (to simulate a bigger vessel).
    Length is the number of points in the vessel.

    The vessel can split in two at each point with a certain probability.
    When the vessel split, the length of the new vessel is half the length of the original vessel.
    Same for the original vessel.
    """
    if submatrix_shape is None:
        submatrix_shape = int(np.log(length)) * 4 + 1
        submatrix_shape = (submatrix_shape, submatrix_shape, submatrix_shape)

    i = 0
    while i < length:
        next_point = extend_vessel(m, x, y, z, distance, submatrix_shape)
        x, y, z = next_point

        # chance to split the vessel
        if np.random.uniform(0, 1) < 0.01:
            print("split ", length - i)
            i += (length - i) // 2
            creat_vessel(m, x + 1, y - 2, z, distance, submatrix_shape, (length - i) // 2)
        i += 1

In [ ]:
m = sparray(shape=(10000, 10000, 10000), default=(0, 0, 0))

In [ ]:
next_point = (5001, 5001, 5001)
m[next_point] = (1, 0, 0)

creat_vessel(m, next_point[0], next_point[1], next_point[2], 2, None, 1000)

In [ ]:
%matplotlib notebook
plot_matrix(m, 2)